In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import configparser

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [3]:
sys.path.append('../../src/')
import apdft
from apdft.calculator.mrcc import MrccCalculator

In [ ]:
def get_modepn(coordinates, fn):
    #return np.zeros(len(coordinates))
    density = MrccCalculator._parse_densityfile(fn)
    modepns = []
    for site in coordinates:
        ds = np.linalg.norm(density[:, :3] - site* ( 1 / 0.52917721067), axis=1)
        
        modepns.append(np.sum(density[:, 3] * density[:, 4] / ds))
    return modepns
def read_meta(basepath):
    nuclear_charges, coordinates = apdft.read_xyz('%s/inp.xyz' % basepath)
    config = configparser.ConfigParser()
    config.read('%s/apdft.conf' % basepath)
    includeonly = [int(_) for _ in config['apdft']['apdft_includeonly'].strip('[]').split(', ')]
    return nuclear_charges, coordinates, includeonly
def read_coefficients(basepath, nuclear_charges, coordinates, includeonly):
    N = len(includeonly)
    
    coefficients = np.zeros((1 + N*2 + N * (N - 1), N))
    
    # order 0
    pos = 0
    coefficients[pos, :] = get_modepn(coordinates[includeonly], '%s/QM/order-0/site-all-cc/DENSITY' % basepath)
    pos += 1
    
    # order 1
    for site in includeonly:
        coefficients[pos, :] = get_modepn(coordinates[includeonly], '%s/QM/order-1/site-%d-up/DENSITY' % (basepath,site))
        coefficients[pos+1, :] = get_modepn(coordinates[includeonly], '%s/QM/order-1/site-%d-dn/DENSITY' % (basepath,site))
        pos += 2
    
    # order 2
    for site_i in includeonly:
        for site_j in includeonly:
            if site_j <= site_i:
                continue
            
            coefficients[pos, :] = get_modepn(coordinates[includeonly], '%s/QM/order-2/site-%d-%d-up/DENSITY' % (basepath,site_i, site_j))
            coefficients[pos+1, :] = get_modepn(coordinates[includeonly], '%s/QM/order-2/site-%d-%d-dn/DENSITY' % (basepath,site_i, site_j))
            pos += 2
    
    return coefficients

basepath = 'data/coronene-dimer/mrcc-monomer-fast'
nuclear_charges, coordinates, includeonly = read_meta(basepath)
coefficients = read_coefficients(basepath, nuclear_charges, coordinates, includeonly)

In [98]:
def predict(deltaZ):
    N = len(includeonly)
    
    # build alphas
    alphas = np.zeros((1 + N*2 + N * (N - 1)))
    
    # order 0
    alphas[0] = 1
    
    # order 1
    for siteidx in range(N):
        alphas[1 + siteidx*2] += 5 * deltaZ[siteidx]
        alphas[1 + siteidx*2+1] -= 5 * deltaZ[siteidx]
    
    # order 2
    pos = 1 + N*2 - 2
    for siteidx_i in range(N):
        for siteidx_j in range(siteidx_i, N):
            if siteidx_i != siteidx_j:
                pos += 2
            if deltaZ[siteidx_j] == 0 or deltaZ[siteidx_i] == 0:
                continue
            if includeonly[siteidx_j] > includeonly[siteidx_i]:
                prefactor = 2 * (200/6.) * deltaZ[siteidx_i]*deltaZ[siteidx_j]
                alphas[pos] += prefactor
                alphas[pos+1] += prefactor
                alphas[0] += 2*prefactor
                alphas[1 + siteidx_i*2] -= prefactor
                alphas[1 + siteidx_i*2+1] -= prefactor
                alphas[1 + siteidx_j*2] -= prefactor
                alphas[1 + siteidx_j*2+1] -= prefactor
            if includeonly[siteidx_j] == includeonly[siteidx_i]:
                prefactor = (400/6.) * deltaZ[siteidx_i]*deltaZ[siteidx_j]
                alphas[0] -= 2*prefactor
                alphas[1 + siteidx_i*2] += prefactor
                alphas[1 + siteidx_j*2+1] += prefactor
    
    # calculate energy
    return np.sum(np.multiply(np.outer(alphas, deltaZ), coefficients))